# Tools for Analyzing Time Series of Satellite Imagery (TATSSI)

## Download data from the GEE
TATSSI allows to dowload data products from Google Earth Engine [GEE](https://developers.google.com/earth-engine/datasets/) and create and process time series from different sensors such as:

* Moderate Resolution Imaging Spectroradiometer [MODIS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#MODIS_anchor)
* Visible Infrared Imaging Radiometer Suite [VIIRS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#S-NPPVIIRS_anchor)

Downloading products from the GEE requires a Google Account, please, first regsiter as a user to get the login credentials and follow the instructions to authenticate to the Earth Engine using the [Python API](https://developers.google.com/earth-engine/python_install).

1. Select your are of study, you can save it as a GeoJSON file, e.g. from the [geojson.io](http://geojson.io/) site.
2. For the time being you need to match the official MODIS product bands and the GEE bands since the band names are not consistent.
3. Generate the time series, you can then use the rest of the (awesome) TATSSI tools!

In [ ]:
# Import libraries
import os
import json

import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

from helpers import utils
from TATSSI.download.gee_downloader import export_collection
from TATSSI.time_series.generator import Generator

In [ ]:
# Area of study
with open('cordoba.geojson') as json_file:
    region = json.load(json_file)
    
region = region['features'][0]['geometry']['coordinates']
geometry = ee.Geometry.Polygon(region)
print(geometry)

For this example we will use the MODIS MOD13A2 product. Match the band names from the [LPDAAC](https://lpdaac.usgs.gov/products/mod13a2v006/) and the [GEE](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2).

In [ ]:
# Dictionary with "GEE" : "LPDAAC"
bands = {"DayOfYear" : "1_km_16_days_composite_day_of_the_year",
         "EVI" : "1_km_16_days_EVI",
         "RelativeAzimuth" : "1_km_16_days_relative_azimuth_angle",
         "SummaryQA" : "1_km_16_days_pixel_reliability",
         "sur_refl_b02" : "1_km_16_days_NIR_reflectance",
         "sur_refl_b07" : "1_km_16_days_MIR_reflectance",
         "DetailedQA" : "1_km_16_days_VI_Quality",
         "NDVI" : "1_km_16_days_NDVI",
         "SolarZenith" : "1_km_16_days_sun_zenith_angle",
         "sur_refl_b01" : "1_km_16_days_red_reflectance",
         "sur_refl_b03" : "1_km_16_days_blue_reflectance",
         "ViewZenith" : "1_km_16_days_view_zenith_angle"}

Set the download parameters

In [ ]:
# Dates as YYYY-MM-DD
start_date, end_date = '2015-01-01', '2020-03-31'

# Get MODIS data
product_name = 'MOD13A2'
version = '006'

# Product using GEE nomenclature
product = f'MODIS/{version}/{product_name}'

# Get GEE collection
ee_collection = (ee.ImageCollection(product).
                 filterBounds(geometry).
                 filterDate(start_date, end_date))

# Output directory
output_dir = '/tmp/gls'

Get the data!

In [ ]:
datasets = export_collection(ee_collection, output_dir, bands=bands,
                             product=product, version=version,
                             scale=None, crs=None, region=region,
                             file_per_band=True)

Create TATSSI time series.

In [ ]:
# TATSSI Time Series Generator object
tsg = Generator(source_dir=output_dir, product=product_name,
                version=version, data_format='tif',
                progressBar=None, preprocessed=True)

# Create layerstacks
tsg._Generator__datasets = datasets
for dataset in tsg._Generator__datasets:
    tsg._Generator__generate_layerstack(dataset, 'tif')

# For the associated product layers, decode the corresponding QA bands
tsg._Generator__decode_qa('tif')

In [ ]:
datasets